In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.master("local").appName("Test Spark").config("spark.some.config.option", "some-value").getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
spark

In [21]:
sqlcontext = SQLContext(sc)

In [5]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [29]:
letter_recognition_df = sqlcontext.read.format('com.databricks.spark.csv').options(header = 'true', inferschema = 'true').load('letter-recognition.csv')
feature_columns = ['x-box','y-box','width','high','onpix','x-bar','y-bar','x2bar','y2bar','xybar','x2ybr','xy2br','x-ege','xegvy','y-ege','yegvx']
vector_assembler = VectorAssembler(inputCols = feature_columns, outputCol = 'features')
vectorised_df = vector_assembler.transform(letter_recognition_df).withColumnRenamed('lettr', 'label').select('label', 'features')
vectorised_df.show(10, False)

+-----+----------------------------------------------------------------------+
|label|features                                                              |
+-----+----------------------------------------------------------------------+
|19   |[2.0,8.0,3.0,5.0,1.0,8.0,13.0,0.0,6.0,6.0,10.0,8.0,0.0,8.0,0.0,8.0]   |
|8    |[5.0,12.0,3.0,7.0,2.0,10.0,5.0,5.0,4.0,13.0,3.0,9.0,2.0,8.0,4.0,10.0] |
|3    |[4.0,11.0,6.0,8.0,6.0,10.0,6.0,2.0,6.0,10.0,3.0,7.0,3.0,7.0,3.0,9.0]  |
|13   |[7.0,11.0,6.0,6.0,3.0,5.0,9.0,4.0,6.0,4.0,4.0,10.0,6.0,10.0,2.0,8.0]  |
|6    |[2.0,1.0,3.0,1.0,1.0,8.0,6.0,6.0,6.0,6.0,5.0,9.0,1.0,7.0,5.0,10.0]    |
|18   |[4.0,11.0,5.0,8.0,3.0,8.0,8.0,6.0,9.0,5.0,6.0,6.0,0.0,8.0,9.0,7.0]    |
|1    |[4.0,2.0,5.0,4.0,4.0,8.0,7.0,6.0,6.0,7.0,6.0,6.0,2.0,8.0,7.0,10.0]    |
|0    |[1.0,1.0,3.0,2.0,1.0,8.0,2.0,2.0,2.0,8.0,2.0,8.0,1.0,6.0,2.0,7.0]     |
|9    |[2.0,2.0,4.0,4.0,2.0,10.0,6.0,2.0,6.0,12.0,4.0,8.0,1.0,6.0,1.0,7.0]   |
|12   |[11.0,15.0,13.0,9.0,7.0,13.0,2.0,6.0,2.0,12.0

In [30]:
train_df, test_df = vectorised_df.randomSplit([0.75, 0.25], seed=12345)
train_df.count(), test_df.count()

(14928, 5072)

In [31]:
layers = [16, 8, 4, 26]

In [32]:
multilayer_perceptron_classifier = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)
multilayer_perceptron_classifier_model = multilayer_perceptron_classifier.fit(train_df)

In [33]:
test_predictions_df = multilayer_perceptron_classifier_model.transform(test_df)
print("TEST DATASET PREDICTIONS AGAINST ACTUAL LABEL: ")
test_predictions_df.select("label", "features", "probability", "prediction").show()

TEST DATASET PREDICTIONS AGAINST ACTUAL LABEL: 
+-----+--------------------+--------------------+----------+
|label|            features|         probability|prediction|
+-----+--------------------+--------------------+----------+
|    0|[1.0,0.0,2.0,0.0,...|[0.15368389851585...|       0.0|
|    0|[1.0,1.0,2.0,1.0,...|[0.15585546934780...|       0.0|
|    0|[1.0,1.0,2.0,1.0,...|[0.15642191086169...|       0.0|
|    0|[1.0,1.0,2.0,1.0,...|[0.15659746860594...|       0.0|
|    0|[1.0,1.0,3.0,2.0,...|[0.15699066639209...|       0.0|
|    0|[1.0,3.0,2.0,2.0,...|[0.15808115024074...|       0.0|
|    0|[1.0,3.0,2.0,2.0,...|[0.15712097716961...|       0.0|
|    0|[2.0,1.0,3.0,2.0,...|[0.15698174241012...|       0.0|
|    0|[2.0,1.0,4.0,2.0,...|[0.15692234526692...|       0.0|
|    0|[2.0,2.0,4.0,4.0,...|[0.15699595194503...|       0.0|
|    0|[2.0,3.0,3.0,1.0,...|[0.15697626056547...|       0.0|
|    0|[2.0,3.0,3.0,1.0,...|[0.15702609312631...|       0.0|
|    0|[2.0,3.0,4.0,2.0,...|[0.156981

In [34]:
prediction_and_labels = test_predictions_df.select("prediction", "label")
accuracy_evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
precision_evaluator = MulticlassClassificationEvaluator(metricName="weightedPrecision")
recall_evaluator = MulticlassClassificationEvaluator(metricName="weightedRecall")
print("Accuracy on Test Dataset = %g" % accuracy_evaluator.evaluate(prediction_and_labels))
print("Precision on Test Dataset = %g" % precision_evaluator.evaluate(prediction_and_labels))
print("Recall on Test Dataset = %g" % recall_evaluator.evaluate(prediction_and_labels))

Accuracy on Test Dataset = 0.252169
Precision on Test Dataset = 0.222466
Recall on Test Dataset = 0.252169


In [35]:
new_layers = [16, 16, 12, 26]
new_multilayer_perceptron_classifier = MultilayerPerceptronClassifier(maxIter=400, layers=new_layers, blockSize=128, seed=1234)
new_multilayer_perceptron_classifier_model = new_multilayer_perceptron_classifier.fit(train_df)
new_test_predictions_df = new_multilayer_perceptron_classifier_model.transform(test_df)
print("New Accuracy on Test Dataset = %g" % accuracy_evaluator.evaluate(new_test_predictions_df.select("prediction", "label")))

New Accuracy on Test Dataset = 0.660686


In [27]:
data = sqlcontext.read.csv(path='letter-recognition.csv', header = True, inferSchema = True)

In [26]:
feature_columns = ['Merit Score','SSC','TerminalDegree']
vector_assembler = VectorAssembler(inputCols = feature_columns, outputCol = 'features')
vectorised_df = vector_assembler.transform(data).withColumnRenamed('Offered', 'label').select('label', 'features')
vectorised_df.show(10, False)

+-----+---------------------+
|label|features             |
+-----+---------------------+
|0    |[908.813,594.0,702.0]|
|0    |[678.0,759.0,633.0]  |
|0    |[763.0,866.0,720.0]  |
|0    |[719.0,672.0,672.0]  |
|0    |[1025.0,867.0,966.0] |
|0    |[972.0,932.0,962.0]  |
|0    |[539.0,482.0,509.0]  |
|0    |[863.0,763.0,818.0]  |
|0    |[618.0,677.0,579.0]  |
|0    |[678.0,686.0,631.0]  |
+-----+---------------------+
only showing top 10 rows



In [17]:
train_df, test_df = vectorised_df.randomSplit([0.75, 0.25], seed=12345)
train_df.count(), test_df.count()

(497, 158)

In [24]:
layers = [3, 1]

In [25]:
multilayer_perceptron_classifier = MultilayerPerceptronClassifier(maxIter=10, layers=layers, blockSize=12, seed=1234)
multilayer_perceptron_classifier_model = multilayer_perceptron_classifier.fit(train_df)

Py4JJavaError: An error occurred while calling o343.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 23.0 failed 1 times, most recent failure: Lost task 0.0 in stage 23.0 (TID 19) (DESKTOP-HF2TGGL.Home executor driver): org.apache.spark.SparkException: Failed to execute user defined function (OneHotEncoderModel$$Lambda$3514/312775512: (double, int) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:177)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNex

In [9]:
data.show()

+--------------------+-----------+----------+------+-----------+--------------+----------------+---+--------------+------------------+------------------+------------------+------------------+------------------+-----------------+-------------+-------+
|  Application Status|Merit Score|  Domicile|Gender|   Province|Admission Test|Admission Test 2|SSC|TerminalDegree|Optional Subject 1|Optional Subject 2|Optional Subject 3|Optional Subject 4|Optional Subject 5|Document Verified|English Marks|Offered|
+--------------------+-----------+----------+------+-----------+--------------+----------------+---+--------------+------------------+------------------+------------------+------------------+------------------+-----------------+-------------+-------+
|Application Received|    908.813|      OPEN|  male|Baluchistan|             0|               0|594|           702|                 0|                 0|                 0|                 0|                 0|                0|            0|     

In [10]:
data2 =data.select(data.TerminalDegree,data.SSC.alias('label'))
train, test = data2.randomSplit([0.9,0.1])

In [11]:
assembler=VectorAssembler().setInputCols(['TerminalDegree',]).setOutputCol('features')
train01 = assembler.transform(train)
train02 = train01.select("features","label")
train02.show(5)

+--------+-----+
|features|label|
+--------+-----+
|   [0.0]|    0|
| [481.0]|  508|
| [483.0]|  652|
| [483.0]|  657|
| [487.0]|  686|
+--------+-----+
only showing top 5 rows



In [12]:
lr = LinearRegression()
model = lr.fit(train02)

In [13]:
test01 = assembler.transform(test)
test02 = test01.select('features', 'label')
test03 = model.transform(test02)
test03.show(5)

+--------+-----+-----------------+
|features|label|       prediction|
+--------+-----+-----------------+
| [520.0]|  581|638.6554922323418|
| [538.0]|  809|649.8554412567817|
| [547.0]|  564|655.4554157690015|
| [552.0]|  631|658.5665127202348|
| [561.0]|  624|664.1664872324546|
+--------+-----+-----------------+
only showing top 5 rows



In [15]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import polyfit

In [24]:
#x=data.TerminalDegree
#y=train02.select('label')

In [25]:
import chart_studio.plotly as py
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=x,
        y=y,
        mode='markers',
        name='Original_Test',
    ))
fig.add_trace(
    go.Scatter(
        x=x,
        y=y_pred,
        name='Predicted'
    ))
fig.update_layout(
    title="Linear Regression",
    xaxis_title="Displacement",
    yaxis_title="Horse Power",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

ValueError: 
    Invalid value of type 'pyspark.sql.column.Column' received for the 'x' property of scatter
        Received value: Column<'TerminalDegree'>

    The 'x' property is an array that may be specified as a tuple,
    list, numpy array, or pandas Series

In [22]:
evaluator = RegressionEvaluator()
print(evaluator.evaluate(test03,
{evaluator.metricName: "r2"})
)
print(evaluator.evaluate(test03,
{evaluator.metricName: "mse"})
)
print(evaluator.evaluate(test03,
{evaluator.metricName: "rmse"})
)
print(evaluator.evaluate(test03,
{evaluator.metricName: "mae"})
)

0.32144533540896403
8277.846726944514
90.9826726742214
71.50585015787924


In [ ]:
#testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())

In [23]:
#print('Test Error = ' + str(testErr))
print('Learned classification tree model:')
print(model.toDebugString())

Learned classification tree model:


AttributeError: 'LinearRegressionModel' object has no attribute 'toDebugString'